In [2]:
import numpy as np
import PIL.Image as Image # Pillow
import os
import tracemalloc
from time import process_time
import math

In [1]:
def load_values(file): # Opening the text file and saving to the corresponding lists.
    val_groundtruth = []
    with open(file, 'r') as f:
        val_set = f.read().splitlines()
    for line in val_set:
        # Image ground truth.
        ground_truth = line.split(' ')[1]
        val_groundtruth.append(int(ground_truth))
    return val_groundtruth


In [3]:
# Image Preprocessing for classification
def preprocessing(image, image_shape): # Inputs loaded image, desired image dimensions.
    input = image.resize(image_shape) # Resizing.
    input = np.array(input)/255.0 # Color range [0,1].
    if(input.shape != (image_shape[0],image_shape[1],3)): # In case the image is black and white.
        input = np.repeat(input[:, :, np.newaxis], 3, axis=2) # We repeat the grayscale 3 times to get the correct dimensions.
    input = input[np.newaxis,...] # Extra axis.
    return input # Returns the preprocessed image

In [2]:
def prepare_standard(im):
    #im = Image.open(path)
    if im.mode == 'CMYK':
        im = im.convert('RGB')
    width, height = im.size
    new_height = height * 256 // min(width,height)
    new_width = width * 256 // min(width,height)
    im = im.resize((new_width,new_height))

    left = (new_width - 224)/2
    top = (new_height - 224)/2
    right = (new_width + 224)/2
    bottom = (new_height + 224)/2

    im = im.crop((left, top, right, bottom))
    input = (np.array(im))/255
    if(input.shape != (224,224,3)): 
        input = np.repeat(input[:, :, np.newaxis], 3, axis=2) 
    #input = input[np.newaxis,...]
    return input

In [4]:
# Ellipse defined by the labels of DAGM
def calc_ellipse(x, y,label):
    # https://www.maa.org/external_archive/joma/Volume8/Kalman/General.html
    ...
    [semi_major, semi_minor, rotation, x_centre, y_centre] = label
    term1 = (((x - x_centre) * np.cos(rotation)) + (
        (y - y_centre) * np.sin(rotation)))**2
    term2 = (((x - x_centre) * np.sin(rotation)) - (
        (y - y_centre) * np.cos(rotation)))**2
    ellipse = ((term1 / semi_major**2) + (term2 / semi_minor**2)) <= 1
    return ellipse

In [5]:
# IoU calculation
def iou_calc(labimg, pred):

    pred = np.round(np.squeeze(pred),decimals = 6).astype(dtype=bool)
    intersection = np.logical_and(labimg, pred)
    union = np.logical_or(labimg, pred)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [3]:
def prepare(path, size):
    im = Image.open(path)
    im = im.convert('RGB')
    re_size = int(round(1.14286*size))
    
    width, height = im.size
    new_height = height * re_size // min(width,height)
    new_width = width * re_size // min(width,height)
    im = im.resize((new_width,new_height))

    left = math.floor((new_width - size)/2)
    top = math.ceil((new_height - size)/2)
    right = math.floor((new_width + size)/2)
    bottom = math.ceil((new_height + size)/2)


    im = im.crop((left, top, right, bottom))
    input = (np.array(im))/255
    return input
